<a href="https://colab.research.google.com/github/TonyN2060/Capstone-Project/blob/main/wav2vec_kiswahili_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prelude

## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Package Installs

In [ ]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install jiwer
!pip install accelerate
!pip install transformers[torch]
!pip install sentencepiece
# !pip install torchaudio
!pip install librosa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.4 MB/s eta 0:00:00


## Package Imports

In [ ]:
import os
import subprocess
import transformers
import datasets
import IPython.display
import numpy
import pandas
import json
import random
import re
import torch
import accelerate
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
import sentencepiece
import torchaudio
import librosa

## HuggingFace API

In [ ]:
!git config --global credential.helper store

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Dataset Imports

In [ ]:
training_set = datasets.load_dataset("mozilla-foundation/common_voice_13_0", "sw", split="train[:-25%]")
eval_set = datasets.load_dataset("mozilla-foundation/common_voice_13_0", "sw", split="validation[:-25%]")

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 11676it [00:00, 116751.76it/s]
Reading metadata...: 34980it [00:00, 137255.33it/s]


Generating validation split: 0 examples [00:00, ? examples/s]

...


Reading metadata...: 11271it [00:00, 158679.65it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 15795it [00:00, 157934.71it/s]
Reading metadata...: 31589it [00:00, 155271.73it/s]
Reading metadata...: 47120it [00:00, 140487.19it/s]
Reading metadata...: 61311it [00:00, 140165.02it/s]
Reading metadata...: 75412it [00:00, 139981.92it/s]
Reading metadata...: 89678it [00:00, 140858.29it/s]
Reading metadata...: 104246it [00:00, 142390.07it/s]
Reading metadata...: 119667it [00:00, 146091.25it/s]
Reading metadata...: 134660it [00:00, 147273.76it/s]
Reading metadata...: 149818it [00:01, 148589.94it/s]
Reading metadata...: 164837it [00:01, 149074.94it/s]
Reading metadata...: 179755it [00:01, 141745.07it/s]
Reading metadata...: 194197it [00:01, 142520.33it/s]
Reading metadata...: 208508it [00:01, 141852.42it/s]
Reading metadata...: 222734it [00:01, 141195.64it/s]
Reading metadata...: 236881it [00:01, 140511.00it/s]
Reading metadata...: 250951it [00:01, 137210.05it/s]
Reading metadata...: 265738it [00:01, 140316.93it/s]
Reading met

Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 14701it [00:00, 146997.52it/s]
Reading metadata...: 30312it [00:00, 152351.08it/s]
Reading metadata...: 46440it [00:00, 156423.34it/s]
Reading metadata...: 64047it [00:00, 152583.69it/s]


# Processor

# Tokenizer

In [ ]:
training_set = training_set.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
eval_set = eval_set.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [ ]:
training_set

Dataset({
    features: ['path', 'audio', 'sentence', 'variant'],
    num_rows: 26235
})

## Random Sampler

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

# For reproducibility https://web.archive.org/web/20220719020554/https://towardsdatascience.com/how-to-use-random-seeds-effectively-54a4cd855a79
                # TODO: Compare seed techniques from https://pieriantraining.com/how-to-set-the-seed-in-pytorch-for-reproducible-results/

seed = 42
torch.manual_seed(seed)


def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(training_set.remove_columns(["path","audio"]), num_examples=10)

,sentence,variant
0,alitelekezwa na mume wake wakati wa kesi,
1,Kimila malighafi zinazozalishwa kienyeji kama mbegu ngozi shaba na mfupa,
2,Kauli ya Profesa Tibaijuka ilitoka siku chache baada ya kujiuzulu,
3,Biashara ya utumwa iliwaweka watu pamoja,
4,Alirejea ahadi yake kwa wananchi mbele ya Baraza hilo,
5,Huongeza hamu ya kula.,
6,Ufundishaji wa fasihi ya watoto unaonekana kuachwa nyuma.,
7,Kazi yao kubwa ilikuwa ni kutabiri mambo ya baadaye katika jamii,
8,Bao lake la kwanza alifunga dhidi ya Chelsea kwenye ligi ya mabingwa Ulaya.,
9,Baada ya kupata nafasi hiyo Machel alikataa kuwapa Wareno makubaliano,


## Vocab list - RAW

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = training_set.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=training_set.column_names)
vocab_test = eval_set.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=eval_set.column_names)

Map:   0%|          | 0/26235 [00:00<?, ? examples/s]

Map:   0%|          | 0/8456 [00:00<?, ? examples/s]

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 '!': 1,
 '"': 2,
 "'": 3,
 '(': 4,
 ')': 5,
 '*': 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '/': 10,
 ':': 11,
 ';': 12,
 '?': 13,
 'A': 14,
 'B': 15,
 'C': 16,
 'D': 17,
 'E': 18,
 'F': 19,
 'G': 20,
 'H': 21,
 'I': 22,
 'J': 23,
 'K': 24,
 'L': 25,
 'M': 26,
 'N': 27,
 'O': 28,
 'P': 29,
 'Q': 30,
 'R': 31,
 'S': 32,
 'T': 33,
 'U': 34,
 'V': 35,
 'W': 36,
 'X': 37,
 'Y': 38,
 'Z': 39,
 '_': 40,
 'a': 41,
 'b': 42,
 'c': 43,
 'd': 44,
 'e': 45,
 'f': 46,
 'g': 47,
 'h': 48,
 'i': 49,
 'j': 50,
 'k': 51,
 'l': 52,
 'm': 53,
 'n': 54,
 'o': 55,
 'p': 56,
 'q': 57,
 'r': 58,
 's': 59,
 't': 60,
 'u': 61,
 'v': 62,
 'w': 63,
 'x': 64,
 'y': 65,
 'z': 66,
 'µ': 67,
 'Ã': 68,
 'Å': 69,
 'á': 70,
 'â': 71,
 'è': 72,
 'é': 73,
 'ë': 74,
 'í': 75,
 'ñ': 76,
 'ó': 77,
 'ö': 78,
 'ø': 79,
 'ú': 80,
 'Š': 81,
 'ū': 82,
 'ː': 83,
 'μ': 84,
 'ụ': 85,
 '‘': 86,
 '’': 87,
 '“': 88,
 '”': 89,
 '…': 90}

## Replace foreign chars and UPPERCASE letters

In [ ]:
def replace_foreign_chars(batch):
    batch["sentence"] = re.sub('[Ã]', 'A', batch["sentence"])
    batch["sentence"] = re.sub('[â]', 'a', batch["sentence"])
    batch["sentence"] = re.sub('[ã]', 'a', batch["sentence"])
    batch["sentence"] = re.sub('[ë]', 'e', batch["sentence"])
    batch["sentence"] = re.sub('[μ]', 'u', batch["sentence"])
    batch["sentence"] = re.sub('[ó]', 'o', batch["sentence"])
    batch["sentence"] = re.sub('[è]', 'e', batch["sentence"])
    batch["sentence"] = re.sub('[ó]', 'o', batch["sentence"])
    batch["sentence"] = re.sub('[ú]', 'u', batch["sentence"])
    batch["sentence"] = re.sub('[í]', 'i', batch["sentence"])
    batch["sentence"] = re.sub('[Å]', 'A', batch["sentence"])
    batch["sentence"] = re.sub('[á]', 'a', batch["sentence"])
    batch["sentence"] = re.sub('[é]', 'e', batch["sentence"])
    batch["sentence"] = re.sub('[ö]', 'o', batch["sentence"])
    batch["sentence"] = re.sub('[ø]', 'o', batch["sentence"])
    batch["sentence"] = re.sub('[Š]', 'S', batch["sentence"])
    batch["sentence"] = re.sub('[ū]', 'u', batch["sentence"])
    batch["sentence"] = re.sub('[ụ]', 'u', batch["sentence"])
    batch["sentence"] = re.sub('[ñ]', 'n', batch["sentence"])
    batch["sentence"] = re.sub('[µ]', 'm', batch["sentence"])

    return batch

In [ ]:
training_set = training_set.map(replace_foreign_chars)
eval_set = eval_set.map(replace_foreign_chars)

Map:   0%|          | 0/26235 [00:00<?, ? examples/s]

Map:   0%|          | 0/8456 [00:00<?, ? examples/s]

In [ ]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\’\‘\…\(\)\*\/\_\ː]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch

In [ ]:
training_set = training_set.map(remove_special_characters)
eval_set = eval_set.map(remove_special_characters)

Map:   0%|          | 0/26235 [00:00<?, ? examples/s]

Map:   0%|          | 0/8456 [00:00<?, ? examples/s]

In [ ]:
show_random_elements(training_set.remove_columns(["path","audio"]), num_examples=10)

,sentence,variant
0,mashariki kawaida huwa upande wa kulia kwenye ramani,
1,kwa kushinda vita hivyo ambavyo mwenyewe ameviita,
2,halmashauri ya jiji la dar es salaam imekuwa inaliongoza jiji,
3,nikisema chungu nke ni kula na nguvo na chungu ni nkaragadho,
4,kenya na buganda ikiwemo na tanganyika zilikabidhiwa kwa ujerumani,
5,katika taifa hilo linaloandamwa na ugonjwa wa kipindupindu,
6,wakati kirumi alipopelekwa katika kukamata kwake,
7,rugemba ndipo lilipo kaburi la mtwa munyigumba ignas pangiligosi muyinga maarufu,
8,real madrid ndio timu tajiri zaidi duniani zaidi ya miongo miwili,
9,bahari haina kina kubwa lakini kuna usafiri mwingi wa meli za kila aina,


### Character extract

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = training_set.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=training_set.column_names)
vocab_test = eval_set.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=eval_set.column_names)

Map:   0%|          | 0/26235 [00:00<?, ? examples/s]

Map:   0%|          | 0/8456 [00:00<?, ? examples/s]

### Vocab list - FINAL

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

29

In [ ]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)
    # saves to /content/vocab.json

In [ ]:
tokenizer = transformers.Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

## Feature Extractor

In [ ]:
feature_extractor = transformers.Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
processor = transformers.Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

# Dataset Prep

## `cast_column` method - sampling rate

In [ ]:
training_set = training_set.cast_column("audio", datasets.Audio(sampling_rate=16000))
eval_set = eval_set.cast_column("audio", datasets.Audio(sampling_rate=16000))

## Clean sets, specific per import

In [ ]:
training_set_clean = training_set
eval_set_clean = eval_set

In [ ]:
training_set = training_set_clean
eval_set = eval_set_clean

## Augmentation

In [ ]:
# Function to apply audio augmentation
def augment_audio(example):
    waveform, sample_rate = torchaudio.load(example['path'])  # Load audio file using the 'path' feature

    # Applying random noise
    noise = torch.randn(waveform.size())
    waveform_noisy = waveform + 0.05 * noise  # arbitrary

    # Applying random time stretching
    # rate = random.uniform(0.8, 1.2)  # Change the stretching rate within the given range
    # waveform_stretched = torchaudio.transforms.Resample(sample_rate, int(sample_rate * rate))(waveform_noisy)

    return {
      'path': example['path'],  # Keep the original audio file path
      'audio': waveform_noisy  # Return augmented waveform
         }



# Applying augmentation to the dataset
# augmented_training_set = training_set.map(augment_audio)
training_set = training_set.map(augment_audio)

Map:   0%|          | 0/26235 [00:00<?, ? examples/s]

## Random Sampling

In [ ]:
rand_int = random.randint(0, len(training_set)-1)

print("Target text:", training_set[rand_int]["sentence"])
print("Input array shape:", training_set[rand_int]["audio"]["array"].shape)
print("Sampling rate:",training_set[rand_int]["audio"]["sampling_rate"])

Target text: mawaziri hawa wa uchumi masuala ya mambo ya nje na kilimo
Input array shape: (78336,)
Sampling rate: 16000


## `prepare_dataset` function with `batch` argument

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [ ]:
training_set.features

{'path': Value(dtype='string', id=None),
 'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None),
 'sentence': Value(dtype='string', id=None),
 'variant': Value(dtype='string', id=None)}

## `map` method

In [ ]:
training_set = training_set.map(prepare_dataset, remove_columns=training_set.column_names, num_proc=4)
eval_set = eval_set.map(prepare_dataset, remove_columns=eval_set.column_names, num_proc=4)

Map (num_proc=4):   0%|          | 0/26235 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

Map (num_proc=4):   0%|          | 0/8456 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

In [ ]:
# training_set_xl = training_set.map(prepare_dataset, num_proc=4)
# eval_set_xl = eval_set.map(prepare_dataset, num_proc=4)

# training_set_xxl = training_set.map(prepare_dataset, remove_columns=training_set.column_names, num_proc=4)
# eval_set_xxl = eval_set.map(prepare_dataset, remove_columns=eval_set.column_names, num_proc=4)

In [ ]:
# print(training_set.features)
# print(eval_set_xl.features)

{'path': Value(dtype='string', id=None), 'audio': Sequence(feature=Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), length=-1, id=None), 'sentence': Value(dtype='string', id=None), 'variant': Value(dtype='string', id=None)}
{'path': Value(dtype='string', id=None), 'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None), 'sentence': Value(dtype='string', id=None), 'variant': Value(dtype='string', id=None)}


In [ ]:
print(training_set.features)
print(eval_set.features)

{'input_values': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}
{'input_values': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}


# Data Collator

In [ ]:
@dataclass
class DataCollatorCTCWithPadding:
    processor: transformers.Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

# WER Metric

## `load_metric`

In [ ]:
wer_metric = datasets.load_metric("wer")

<ipython-input-47-76b13ab03f66>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = datasets.load_metric("wer")


## `compute_metrics`

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = numpy.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

# Model

## Definition

In [ ]:
model_path = "/content/drive/MyDrive/xlsr-v-model/"
model = transformers.Wav2Vec2ForCTC.from_pretrained(model_path)

In [ ]:
type(model)

transformers.models.wav2vec2.modeling_wav2vec2.Wav2Vec2ForCTC

`freeze_feature_extractor`

In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1913: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


## `gradient_checkpointing_enable`

In [ ]:
model.gradient_checkpointing_enable()

## Accelerator with `accelerate.prepare`

In [ ]:
accelerator = accelerate.Accelerator()

In [ ]:
model = accelerator.prepare(model)

# Training



## `TrainingArguments`

In [ ]:
training_args = transformers.TrainingArguments(
  output_dir="/content/drive/MyDrive/xlsr-v-model-2",
  group_by_length=True, # `True` groups samples of similar lengths together during training, improves training efficiency by reducing padding.
  per_device_train_batch_size=16, # batch size per device for training; each GPU/TPU will process 16 samples in parallel during each forward and backward pass.
  gradient_accumulation_steps=2, # accumulate gradients over multiple steps before performing a backward pass; useful with limited memory and want to use a larger effective batch size.
  evaluation_strategy="steps", # evaluation will be performed after a certain number of steps.
  num_train_epochs=30, # total number of training epochs. An epoch is one pass through the entire training dataset.
  fp16=True, # enables mixed-precision training, which uses lower-precision data types (e.g., float16) for certain parts of the training process to reduce memory usage and potentially speed up training.
  save_steps=100, # how often the model and optimizer state should be saved during training
  eval_steps=100, # specifies how often to perform evaluation steps.
  logging_steps=10, # how often logging information (such as training loss) should be printed during training.
  learning_rate=3e-4, # sets the initial learning rate for the optimizer.
  warmup_steps=500, # controls the number of warm-up steps during learning rate scheduling. Warm-up steps gradually increase the learning rate at the beginning of training.
  save_total_limit=2, # will limit the total amount of checkpoints. Deletes the older checkpoints in increments of save_steps.
  seed=seed

)

## Trainer

In [ ]:
trainer = transformers.Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=training_set,
    eval_dataset=eval_set,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
100,0.326600,0.327305,0.999754
200,0.344800,0.324293,0.996070
300,0.311200,0.326343,0.987895
400,0.330300,0.332612,0.996905
500,0.342600,0.373641,1.008435
600,0.334000,0.341549,1.005902


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Tran

KeyboardInterrupt: ignored

# Evaluation

In [ ]:
try:
    trainer.evaluate(eval_set)
except tf.errors.FailedPreconditionError as e:
    print("FailedPreconditionError:", e)
    trainer.evaluate(eval_set)

except tf.errors.FailedPreconditionError as e:
    print("FailedPreconditionError:", e)


Step,Training Loss,Validation Loss,Wer
100,3.793400,3.462703,1.000000
200,2.938400,2.910155,1.000000
300,2.855600,2.880993,1.000000
400,2.744800,2.690972,1.000000
500,0.907200,0.803234,0.722714
600,0.724100,0.623740,0.614660
700,0.604100,0.540798,0.511277
800,0.557300,0.479059,0.457511
900,0.373800,0.448286,0.436045
1000,0.374100,0.423372,0.399114


FailedPreconditionError: ignored

In [ ]:
model_path = "/content/drive/MyDrive/xlsr-v-model"
model.save_pretrained(model_path)

In [ ]:
test_set = datasets.load_dataset("mozilla-foundation/common_voice_13_0", "sw", split="test[:25%]")

In [ ]:
test_set = test_set.cast_column("audio", datasets.Audio(sampling_rate=16000))

In [ ]:
import pandas as pd
import csv
import torchaudio
output_file = "transcriptions.csv"

with open(output_file, 'w', newline='') as csvfile:
    fieldnames = ['audio_path', 'transcription']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for example in test_set:
        audio_file_path = example["path"]

        # Load and process the audio file
        waveform, sample_rate = torchaudio.load(audio_file_path)
        # Resample or split audio if needed to fit the model requirements

        # Preprocess the audio to fit the model's requirements
        inputs = processor(waveform.squeeze(), sampling_rate=16000, return_tensors="pt", padding=True)
        inputs = inputs.input_values.to(torch.float16).cuda()  # Convert to model's expected data type

        with torch.no_grad():
             logits = model(input_values=inputs).logits

        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.batch_decode(predicted_ids)[0]

        # Write to CSV
        writer.writerow({'audio_path': audio_file_path, 'transcription': transcription})

print(f"CSV file generated: {output_file}")

CSV file generated: transcriptions.csv


In [ ]:
df = pd.read_csv(output_file)
df

,audio_path,transcription
0,/root/.cache/huggingface/datasets/downloads/ex...,bo aritarisia rianucho hii hiku wa chuko bai o...
1,/root/.cache/huggingface/datasets/downloads/ex...,wilalingaka mayaja ya chasaka anaseka haha ya ...
2,/root/.cache/huggingface/datasets/downloads/ex...,shuo tavaashomee haapiwaamiovalida vinjani hetee
3,/root/.cache/huggingface/datasets/downloads/ex...,majurikana kamashina na mworona
4,/root/.cache/huggingface/datasets/downloads/ex...,fueunvo ako midakanakaza viwa palimaofana
...,...,...
2813,/root/.cache/huggingface/datasets/downloads/ex...,na haba uzingia usmakame sholo hooo
2814,/root/.cache/huggingface/datasets/downloads/ex...,asatendi huo nashona hata yavoliayo sakta mshw...
2815,/root/.cache/huggingface/datasets/downloads/ex...,tishina zitwerego mbacho lidoho dma mwanao ouc...
2816,/root/.cache/huggingface/datasets/downloads/ex...,omberi imagotwonambe nirakia kinwoni
